In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cvxpy
import time

import multiprocessing as mp
from tqdm import tqdm

In [32]:
import io_utils
import kernels
import algos
from importlib import reload
reload(io_utils)

<module 'io_utils' from '/home/victor/Data/Mines-ParisTech/M2/mva/S2/km/kernel-methods/io_utils.py'>

In [33]:
np.random.seed(42)

raw_train, raw_val, labels_train, labels_val, feats_train, feats_val = [], [], [], [], [], []

for i in range(3):
    r_tr, r_val, feats_tr, feats_v, lbl_tr, lbl_val = io_utils.train_val_single_dataset(i)
    raw_train.append(r_tr)
    raw_val.append(r_val)
    labels_train.append(lbl_tr)
    labels_val.append(lbl_val)
    feats_train.append(feats_tr)
    feats_val.append(feats_v)

In [75]:
pool = mp.Pool(6)
# Compute kernels
kernel_train, kernel_vals = [], []

kern_ss = kernels.SubstringKernel(7, 0.1)

for n in range(3):
    print(n)
    n_train, n_val = raw_train[n].shape[0], raw_val[n].shape[0]
    
    t = time.time()
    args = [(raw_train[n][i], raw_train[n][j]) for i in range(n_train) for j in range(i, n_train) ]
    result = pool.map(kern_ss.computeKernel, args)
    e = time.time()
    print(e-t)
    
    count = 0
    K = np.zeros((n_train, n_train))
    for i in range(n_train):
        for j in range(i, n_train):
            K[i,j] = result[count]
            count +=1

    for i in range(n_train):
        for j in range(0, i):
            K[i,j] = K[j,i]
  
    kernel_train.append(K.copy())
    args = [(raw_train[n][i], raw_val[n][j]) for i in range(n_train) for j in range(n_val) ]
    
    result = pool.map(kern_ss.computeKernel, args)
    intermediate = np.zeros((n_train, n_val))
    count = 0
    for i in tqdm(range(n_train)):
        for j in range(n_val):
            intermediate[i,j] = result[count]
            count +=1
    kernel_vals.append(intermediate.copy())



0


Process ForkPoolWorker-134:
Process ForkPoolWorker-129:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-131:
Process ForkPoolWorker-130:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Process ForkPoolWorker-132:
Traceback (most recent call last):


KeyboardInterrupt: 

Process ForkPoolWorker-133:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.

In [36]:
for i in range(3):
    np.savetxt('kernels/substring_3_01_train_train_{}.txt'.format(i), kernel_train[i])
    np.savetxt('kernels/substring_3_01_train_val_{}.txt'.format(i), kernel_vals[i])
    np.savetxt('kernels/substring_3_01_labels_train_{}.txt'.format(i), labels_train[i])
    np.savetxt('kernels/substring_3_01_labels_val_{}.txt'.format(i), labels_val[i])
    with open('kernels/substring_3_01_raw_train_{}.txt'.format(i), 'w') as f:
        f.write("\n".join(raw_train[i]))
    
    with open('kernels/substring_3_01_raw_train_{}.txt'.format(i), 'w') as f:
        f.write("\n".join(raw_val[i]))

In [77]:
for i in range(3):
    np.savetxt('kernels/substring_3_01_feats_train_{}.txt'.format(i), feats_train[i])
    np.savetxt('kernels/substring_3_01_feats_val_{}.txt'.format(i), feats_val[i])


In [5]:
# Test
kernel_tests = []

for i in range(3):
    cur_test = io_utils.load_raw_test(i)
    
    args = [(raw_train[i][j], cur_test[k]) for j in range(raw_train[i].shape[0]) for k in range(cur_test.shape[0]) ]
    result = pool.map(computeKernel, args)

    kernel_test = np.zeros((raw_train[i].shape[0], cur_test.shape[0]))

    count = 0
    for k in range(raw_train[i].shape[0]):
        for l in range(cur_test.shape[0]):
            kernel_test[k,l] = result[count]
            count +=1
            
    kernel_tests.append(kernel_test.copy())

NameError: name 'computeKernel' is not defined

In [ ]:
# Load from files
# kernel_train, labels_train, kernel_vals, labels_vals, kernel_tests = io_utils.load_kernels("kernels/substring_7_01")

### Linear kernel

In [54]:
linear_kernel = kernels.LinearKernel()
K_linear_train, K_linear_val = [], []
for i in range(3):
    K_linear_train.append(linear_kernel.computeVectorizedKernel(feats_train[i], feats_train[i]))
    K_linear_val.append(linear_kernel.computeVectorizedKernel(feats_train[i], feats_val[i]))    

In [74]:
## Prediction 
lambd = 7e-1
lambd2 = 7e-6
TRIALS = 30

lambda_vals = np.logspace(-7, -3, TRIALS)

all_preds = []

krr = algos.KernelRidgeRegression()

weights = np.ones((3,2))

for i in range(3):
    krr.fit(kernel_train[i], labels_train[i], lambd)
    error_1 = krr.evaluatePerformance(kernel_vals[i], labels_val[i])
    predictions_1 = np.einsum('i, ij->j', krr.alpha, kernel_vals[i])

    krr.fit(K_linear_train[i], labels_train[i], lambd2)
    error_2 = krr.evaluatePerformance(K_linear_val[i], labels_val[i])
    w_1 = (1-error_2)/(1-error_1)
    predictions_2 = np.einsum('i, ij->j', krr.alpha, K_linear_val[i])
    global_error = 1 / (w_1 + 1) * predictions_1 + w_1 / (w_1 + 1) * predictions_2
    print(np.round(1-np.sum(np.sign(global_error) != np.sign(labels_val[i])) / labels_val[i].shape[0],3))
    print("----")

# Test
#     pred_test = krr.predict(kernel_tests[i])
#     all_preds.append(pred_test.copy())

    

Score : 0.605
Score : 0.588
0.628
----
Score : 0.633
Score : 0.573
0.617
----
Score : 0.690
Score : 0.698
0.715
----


In [42]:
TRIALS = 10

lambda_vals = np.logspace(-2, 3, TRIALS)

krr = algos.KernelRidgeRegression()
for i in range(3):
    errors = krr.searchBestParameters(kernel_train[i], train_labels[i], kernel_vals[i], val_labels[i], lambda_vals)

NameError: name 'train_labels' is not defined